A pivot table is a way of summarizing data in a DataFrame for a particular purpose. It makes heavy use of this aggregation function we've been talking about. A pivot table is in itself a DataFrame, where the rows represent one variable that we're interested in, the columns another, and then the cells some aggregate value.

A pivot table also tends to include marginal values as well, which are sum for each column and row. This allows we to be able to see the relationship between two variables.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('resources/week-3/datasets/cwurData.csv')
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


Here we can see that each institutions rank, country, quality of education and other metrics, and overall score are shown. Let's say we wanted to create a new column called Rank_Level, where institutions with world rankings 1-100 are categorized as first tier, and those with world rankings 101 to 200 are second tier, and ranking 201 to 300 are third tier. Then after 301, we'll just bucket those as other top universities.

In [3]:
def create_category(ranking):
    if (ranking >= 1) & (ranking <= 100):
        return 'First Tier Top University'
    elif (ranking >= 101) & (ranking <= 200):
        return 'Second Tier Top University'
    elif (ranking >= 201) & (ranking <= 300):
        return 'Third Tier Top University'
    return 'Other Tier Top University'

# applying this to a single column of data to create a new series
df['Rank_Level'] = df['world_rank'].apply(lambda x: create_category(x))
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,Rank_Level
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012,First Tier Top University
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012,First Tier Top University
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012,First Tier Top University
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012,First Tier Top University
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012,First Tier Top University


A pivot table allows us to pivot out one of these columns headers and compare it against another column as row indices. Let's say we want to compare rank level versus country of the universities and we want to compare in terms of overall score

In [ ]:
# to do this, we set the values to be score, and index to be the country and the columns to be the rank levels
# then we specify the aggregation function, and use numpy mean to get the average rating for universities in that country 

df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean]).head()
# calculating the score mean

So we can see a hierarchical DataFrame, where the index or rows are by country and the columns have two levels. The top level indicating that the mean value is being used and the second level being our ranks. In this example, we only have one variable, the mean, that we're looking at. So we don't really need a hierarchical index.

We notice that there are some Not a Number values. For example, the first row for Argentina. The NaN values indicate that Argentina only has observations in the other top university category. 

In [5]:
# pivot table are not limited to one function that we might want to apply. 
# We can pass a named parameter, aggfunc, which is a list of the different functions to apply, 
# and pandas will provide us with the result using hierarchical column names. 
# Let's try the same query, but pass in the max() also
df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean, np.max]).head()

mean                            \
Rank_Level First Tier Top University Other Tier Top University   
country                                                          
Argentina                        NaN                 44.672857   
Australia                    47.9425                 44.645750   
Austria                          NaN                 44.864286   
Belgium                      51.8750                 45.081000   
Brazil                           NaN                 44.499706   

                                                                 \
Rank_Level Second Tier Top University Third Tier Top University   
country                                                           
Argentina                         NaN                       NaN   
Australia                     49.2425                 47.285000   
Austria                           NaN                 47.066667   
Belgium                       49.0840                 46.746667   
Brazil                        49.5650                       NaN   

                                amax                            \
Rank_Level First Tier Top University Other Tier Top University   
country                                                          
Argentina                        NaN                     45.66   
Australia                      51.61                     45.97   
Austria                          NaN                     46.29   
Belgium                        52.03                     46.21   
Brazil                           NaN                     46.08   

                                                                 
Rank_Level Second Tier Top University Third Tier Top University  
country                                                          
Argentina                         NaN                       NaN  
Australia                       50.40                     47.47  
Austria                           NaN                     47.78  
Belgium                         49.73                     47.14  
Brazil                          49.82                       NaN

Remember we're not invoking the function here, but passing a reference to it and np.max. Now we see that we have both the mean and the max. We can also summarize the values within a given top-level group. 

For instance, if we wanted to see an overall average for the country with the mean and we wanted to see the max of the max, we can indicate that we want Pandas to do this by providing marginal values. Our function looks basically the same. We pass in aggfunc with our two values.

In [7]:
# then we just add an extra parameter to the end called margins equals true.
df.pivot_table(values='score', index='country', columns='Rank_Level',
               aggfunc=[np.mean, np.max], margins=True).head()

mean                            \
Rank_Level First Tier Top University Other Tier Top University   
country                                                          
Argentina                        NaN                 44.672857   
Australia                    47.9425                 44.645750   
Austria                          NaN                 44.864286   
Belgium                      51.8750                 45.081000   
Brazil                           NaN                 44.499706   

                                                                            \
Rank_Level Second Tier Top University Third Tier Top University        All   
country                                                                      
Argentina                         NaN                       NaN  44.672857   
Australia                     49.2425                 47.285000  45.825517   
Austria                           NaN                 47.066667  45.139583   
Belgium                       49.0840                 46.746667  47.011000   
Brazil                        49.5650                       NaN  44.781111   

                                amax                            \
Rank_Level First Tier Top University Other Tier Top University   
country                                                          
Argentina                        NaN                     45.66   
Australia                      51.61                     45.97   
Austria                          NaN                     46.29   
Belgium                        52.03                     46.21   
Brazil                           NaN                     46.08   

                                                                        
Rank_Level Second Tier Top University Third Tier Top University    All  
country                                                                 
Argentina                         NaN                       NaN  45.66  
Australia                       50.40                     47.47  51.61  
Austria                           NaN                     47.78  47.78  
Belgium                         49.73                     47.14  52.03  
Brazil                          49.82                       NaN  49.82

In [7]:
# a pivot table is just a multi-level dataframe, and we can access series or cells in the dataframe in a similar way 
# as we do so for a regular dataframe  

# creating a new dataframe from our previous sample
new_df = df.pivot_table(values='score', index='country', columns='Rank_Level',
                        aggfunc=[np.mean, np.max], margins=True)
print(new_df.index)
print(new_df.columns)

Index(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Bulgaria',
       'Canada', 'Chile', 'China', 'Colombia', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Egypt', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Iran',
       'Ireland', 'Israel', 'Italy', 'Japan', 'Lebanon', 'Lithuania',
       'Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland',
       'Portugal', 'Puerto Rico', 'Romania', 'Russia', 'Saudi Arabia',
       'Serbia', 'Singapore', 'Slovak Republic', 'Slovenia', 'South Africa',
       'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand',
       'Turkey', 'USA', 'Uganda', 'United Arab Emirates', 'United Kingdom',
       'Uruguay', 'All'],
      dtype='object', name='country')
MultiIndex([('mean',  'First Tier Top University'),
            ('mean',  'Other Tier Top University'),
            ('mean', 'Second Tier Top University'),
            ('mean',  'Thir

We can see that the columns are hierarchical. The top-level column indices have two categories, mean and max. The lower level column indices have four categories which are the four rank levels.

In [9]:
# How would we query this if we wanted to get the average scores of the first top tier university levels in each country? 
# We just need to make to dataframe projections. The first for the mean and the second for the top tier.

# 因為是 hierarchical index，故 index 有兩個
new_df['mean']['First Tier Top University'].head()

country
Argentina        NaN
Australia    47.9425
Austria          NaN
Belgium      51.8750
Brazil           NaN
Name: First Tier Top University, dtype: float64

In [10]:
# we can see that the output is a series object. 
# Remember that when we project a single column of values out of a dataframe, we get a series
type(new_df['mean']['First Tier Top University'])

pandas.core.series.Series

In [11]:
# what if we want to find the country that has the maximum average score on First Tier Top University level?
# we can use the idxmax() function for series
new_df['mean']['First Tier Top University'].idxmax()

'United Kingdom'

The idxmax function isn't a special value for pivot tables. It's a built-in function to the series object. 

If we wanted to achieve a different shape of your pivot table, we can do so with the stack and unstack functions. Stacking is pivoting the lowest column index to become the innermost row index, and unstacking is just the inverse of stacking, pivoting the innermost row index to become the lowermost column index. 

In [12]:
# let's look at our pivot table
new_df.head()

mean                            \
Rank_Level First Tier Top University Other Tier Top University   
country                                                          
Argentina                        NaN                 44.672857   
Australia                    47.9425                 44.645750   
Austria                          NaN                 44.864286   
Belgium                      51.8750                 45.081000   
Brazil                           NaN                 44.499706   

                                                                            \
Rank_Level Second Tier Top University Third Tier Top University        All   
country                                                                      
Argentina                         NaN                       NaN  44.672857   
Australia                     49.2425                 47.285000  45.825517   
Austria                           NaN                 47.066667  45.139583   
Belgium                       49.0840                 46.746667  47.011000   
Brazil                        49.5650                       NaN  44.781111   

                                amax                            \
Rank_Level First Tier Top University Other Tier Top University   
country                                                          
Argentina                        NaN                     45.66   
Australia                      51.61                     45.97   
Austria                          NaN                     46.29   
Belgium                        52.03                     46.21   
Brazil                           NaN                     46.08   

                                                                        
Rank_Level Second Tier Top University Third Tier Top University    All  
country                                                                 
Argentina                         NaN                       NaN  45.66  
Australia                       50.40                     47.47  51.61  
Austria                           NaN                     47.78  47.78  
Belgium                         49.73                     47.14  52.03  
Brazil                          49.82                       NaN  49.82

In [13]:
# now try stacking, this should move the lowermost column, the tiers of the university rankings, to the innermost row
new_df = new_df.stack()
new_df.head()

mean   amax
country   Rank_Level                                  
Argentina Other Tier Top University   44.672857  45.66
          All                         44.672857  45.66
Australia First Tier Top University   47.942500  51.61
          Other Tier Top University   44.645750  45.97
          Second Tier Top University  49.242500  50.40

We can see the columns transposed into the rows. In the original pivot table, rank levels are the lowermost column, after stacking, it become the innermost index, appearing to the right after country (從最下面的column變成最右邊的column)

In [14]:
# let's try unstacking
new_df.unstack().head()

mean                            \
Rank_Level First Tier Top University Other Tier Top University   
country                                                          
Argentina                        NaN                 44.672857   
Australia                    47.9425                 44.645750   
Austria                          NaN                 44.864286   
Belgium                      51.8750                 45.081000   
Brazil                           NaN                 44.499706   

                                                                            \
Rank_Level Second Tier Top University Third Tier Top University        All   
country                                                                      
Argentina                         NaN                       NaN  44.672857   
Australia                     49.2425                 47.285000  45.825517   
Austria                           NaN                 47.066667  45.139583   
Belgium                       49.0840                 46.746667  47.011000   
Brazil                        49.5650                       NaN  44.781111   

                                amax                            \
Rank_Level First Tier Top University Other Tier Top University   
country                                                          
Argentina                        NaN                     45.66   
Australia                      51.61                     45.97   
Austria                          NaN                     46.29   
Belgium                        52.03                     46.21   
Brazil                           NaN                     46.08   

                                                                        
Rank_Level Second Tier Top University Third Tier Top University    All  
country                                                                 
Argentina                         NaN                       NaN  45.66  
Australia                       50.40                     47.47  51.61  
Austria                           NaN                     47.78  47.78  
Belgium                         49.73                     47.14  52.03  
Brazil                          49.82                       NaN  49.82

That seems to restore our dataframe to its original shape.

In [15]:
# What would happen if we unstack twice in a row?
new_df.unstack().unstack().head()

      Rank_Level                 country  
mean  First Tier Top University  Argentina        NaN
                                 Australia    47.9425
                                 Austria          NaN
                                 Belgium      51.8750
                                 Brazil           NaN
dtype: float64

We actually end up unstacking all the way to just a single column, so a series object is returned. This column is just a 'value', the meaning of which is denoted by the heirarachical index of operation, rank, and country.

The pivot tables are incredibly useful in dealing with numeric data, especially if we're trying to summarize the data in some form. We'll regularly be creating new pivot tables on different slices of data, whether we're exploring the data ourselves or preparing that data for others to report on. Of course, you can pass any functions we want to the aggregate function, including those that we define ourselves.